# Exploring Page Rank Nibble (Untiled and Runtime Incremental Tiling)

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run prelude.py

#  Create Random Matrix


In [ ]:
import random 

random.seed(0)
def GenerateRandomGraph(numVertex=8, sparsity=25, symmetric=True):
    adjMatrix = [[0 for d in range(numVertex)] for s in range(numVertex)]
    
    genNode = int(numVertex * numVertex * sparsity / 100)
    
    for n in range(genNode):
        while True:
            src = random.randrange(numVertex)
            dst = random.randrange(numVertex)
            if src != dst:
                break
                
        adjMatrix[src][dst] = 1
        if symmetric:
            adjMatrix[dst][src] = 1
    
    return adjMatrix

# Create Random Frontier

In [ ]:
import random 

random.seed(0)
def GenerateRandomFrontier(numVertex=8, frontierSize=8):
    frontier = [ 0 for v in range(numVertex) ]

    for n in range(frontierSize):
        while True:
            v = random.randrange(numVertex)
            if frontier[v] != 1:
                break
        frontier[v] = 1
        
    frontier = Tensor.fromUncompressed(["F"], frontier)
        
    return frontier

# Generate Graph Inputs

In [ ]:
#
# Function to create graph inputs
#


# Create inputs based on the slides that Aamer used to do the initial SPMV mapping on Symphony
def create_inputs(display=True, numNode=8):
    # Adjacency matrix - Ranks "S" (source) and "D" (destination)

    matrix = GenerateRandomGraph(numNode, 25)
    adjMat = Tensor.fromUncompressed([ "S", "D"], matrix)

    # Vector
    srcData = Tensor.fromUncompressed([ "S" ], [ 100 for s in range(numNode) ])

    # Vector
    dstData = Tensor.fromUncompressed([ "D" ], [ 0 for d in range(numNode) ])
    
    if display:
        displayTensor(adjMat)

    if display:
        displayTensor(srcData)

    if display:
        displayTensor(dstData)

    return (dstData, adjMat, srcData)

# Naive Untiled Page Rank Nibble

In [ ]:
V  = 8

#Create inputs
(output, adjMat, input) = create_inputs(False, V)

#Create frontier
frontier = GenerateRandomFrontier(V, 6)

adjMat_s = adjMat.getRoot();
f_s      = frontier.getRoot();
input_v  = input.getRoot();
output_v = output.getRoot();

for s, ((d_fiber, _), input_val) in ((adjMat_s & f_s) & input_v):
    degree   = d_fiber.__len__()
    propVal  = input_val / degree
    for d, (output_ref, _), in (output_v << d_fiber):
        output_ref += propVal

displayTensor(adjMat_s)
displayTensor(f_s)
displayTensor(output_v)

# Runtime Incremental Tiling

In [ ]:
V  = 8

F0 = 2
D0 = 4

print("Tiling Parameters:")
print("\tV  = ", V)
print("\tD0 = ", D0)
print("\tF0 = ", F0)

D1 = int((V/D0+1,V/D0) [(V%D0) == 0])

print("\tD1 = ", D1)

#Create inputs
#(output, adjMat, input) = create_inputs(False, V)

#Create frontier
#frontier = GenerateRandomFrontier(V, 1)

adjMat_s = adjMat.getRoot();
f_s      = frontier.getRoot();
input_v  = input.getRoot();
output_v = output.getRoot();

print("\n\nFrontier: ")
displayTensor(f_s)
frontier_split = f_s.splitEqual(F0)

print("\n\nFrontier Split Equally: ")
displayTensor(frontier_split)

print("\n\nGraph: ")
displayTensor(adjMat_s)

print("\n\nProcessing: ")
# for each equally split frontier tile
for f1, (fr_tile) in (frontier_split):
    
    # for each d1 tile
    for d1 in range(D1):
        
        # Dynamically create a filter fiber for the range of d0 coordinates to process for this d1 tile
        d1_filter = Fiber(coords=[d0 for d0 in range(d1*D0, (d1+1)*D0)], initial=1 )
        
        # for each source (i.e., element of the frontier tile)
        for s, ((d_fiber, _), input_val) in ((adjMat_s & fr_tile) & input_v):
            
            # determine the degree and the value to propogate
            degree   = d_fiber.__len__()
            propVal  = input_val / degree

            # propagate for each destination (that belongs to the current d1 tile)
            for d, (output_ref, _), in (output_v << (d_fiber & d1_filter)):
                #print("Processing: s =", s, " d = ", d)
                output_ref += propVal
                

print("\n\nGraph Output: ")
displayTensor(output_v)